### We are using word embeddings to create positive heuristics to identify positive cases from the proxy statements.
1. Word Embeddings: Word2vec Skipgram model
2. K means clustering using NLTK and Scikit Learn


1. source: https://ai.intelligentonlinetools.com/ml/k-means-clustering-example-word2vec/
2. source: https://github.com/gaetangate/word2vec-cluster
3. source: file:///Users/annamontserratguerrero/Downloads/SSRN-id3480902.pdf
4. source: https://www.cs.toronto.edu/~lczhang/360/lec/w06/w2v.html\
5. source: http://xplordat.com/2018/09/27/word-embeddings-and-document-vectors-part-1-similarity/
6. source: https://srijithr.gitlab.io/post/word2vec/ (Skipgram and pytorch with kmeans)


Things to try: GloVe Embeddings

In [9]:
from tqdm import tqdm
tqdm.pandas()

In [1]:
import string
#from collections import Counter
#import pickle
#import joblib

import numpy as np
import pandas as pd
import nltk
import re
import os

#from sklearn.metrics.pairwise import cosine_similarity
#from matplotlib import pyplot as plt

#%matplotlib inline 


#from nltk.stem.snowball import SnowballStemmer
#from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
#from nltk.corpus import stopwords
#from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
#from sklearn.preprocessing import normalize


from sklearn.cluster import KMeans
#from sklearn.manifold import MDS

In [2]:
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
#RUN
documents = pd.read_csv("def14a_para_clean.csv")
#documents = pd.read_csv("text_ind.csv")
print(documents.shape)
documents.head()

(406160, 4)


,Unnamed: 0,file_name,text,clean_text
0,37,_UNK_DEF 14A_20181110_0001213900-18-013745.txt,"The Board of Directors of FlexShopper, Inc., ...",board director flexshopp delawar corpor compan...
1,39,_UNK_DEF 14A_20181110_0001213900-18-013745.txt,We have established the close of business on ...,establish close busi septemb record date deter...
2,40,_UNK_DEF 14A_20181110_0001213900-18-013745.txt,The Company's Board of Directors recommends t...,compani board director recommend stockhold con...
3,50,_UNK_DEF 14A_20181110_0001213900-18-013745.txt,This Consent Solicitation Statement is being ...,consent solicit statement furnish connect soli...
4,52,_UNK_DEF 14A_20181110_0001213900-18-013745.txt,The Company has decided to seek the written c...,compani decid seek written consent stockhold c...


In [4]:
def preprocess(paragraph):
    text = re.sub(r'\[[0-9]*\]',' ',paragraph)
    text = re.sub(r'\s+',' ',text)
    text = text.lower()
    text = re.sub(r'\d',' ',text)
    text = re.sub(r'\s+',' ',text)
    return text

documents['trial'] = documents.apply(lambda x: preprocess(x['text']), axis = 1)
documents['trial'] = documents.apply(lambda x: nltk.sent_tokenize(x['trial']), axis = 1)

In [5]:
documents.trial

0         [ the board of directors of flexshopper, inc.,...
1         [ we have established the close of business on...
2         [ the company's board of directors recommends ...
3         [ this consent solicitation statement is being...
4         [ the company has decided to seek the written ...
                                ...                        
406155    [if you would like to reduce the costs incurre...
406156    [ please sign exactly as your name(s) appear(s...
406157    [ please sign exactly as your name(s) appear(s...
406158    [the stockholder(s) hereby appoint george j. d...
406159    [this proxy, when properly executed, will be v...
Name: trial, Length: 406160, dtype: object

In [6]:
#each row is a list of sentences. need to go through the list in a for loop and run word_tokenize
#remove punctuation

import string

def token_words(sentence_list): #sentence_list is a row in the datframe
    punctuation=nltk.word_tokenize(string.punctuation)
    sentence_list = [nltk.word_tokenize(sentence) for sentence in sentence_list]
    return [token for token in sentence_list if token not in punctuation]

documents['trial'] = documents.apply(lambda x: token_words(x['trial']), axis = 1)
    

### Remove stopwords is OPTIONAL. I ended up not using this part.
- This paper only removed punctuation: https://academic.oup.com/jamiaopen/article/2/2/246/5423083#198263409

In [15]:
#You can Skip this.
def remove_stopwords(sentences):
    for i in range(len(sentences)): #sentences is one row in trial
        sentences[i] = [word for word in sentences[i] if word not in stopwords.words('english')]
    return sentences
#documents.trial[0]
documents['trial'] = documents.progress_apply(lambda x: remove_stopwords(x['trial']), axis = 1)

100%|██████████| 406160/406160 [2:46:55<00:00, 40.55it/s]      


### Word2Vec needs a list of lists as input:

In [10]:
#turn the rows into a combine nested list
sentences = documents.trial[0]
for row in tqdm(documents.trial):
    sentences += row
 

100%|██████████| 406160/406160 [00:00<00:00, 1817303.54it/s]


https://kavita-ganesan.com/gensim-word2vec-tutorial-starter-code/#.XyzopRNKgUo

Behind the scenes, what’s happening here is that we are training a neural network with a single hidden layer where we train the model to predict the current word based on the context (using the default neural architecture). However, we are not going to use the neural network after training! Instead, the goal is to learn the weights of the hidden layer. These weights are essentially the word vectors that we’re trying to learn. The resulting learned vector is also known as the embeddings. You can think of these embeddings as some features that describe the target word. For example, the word `king` may be described by the gender, age, the type of people the king associates with, etc.

Build a vocabulary and train the model.

### Word2Vec

In [1]:
import gensim

In [3]:
from gensim.models import Word2Vec
#need to find a way to go through each row in the dataframe
#model = Word2Vec(allrows, min_count=1)
model = Word2Vec(sentences, size=150, window=10, min_count=2, workers=10, sg = 1)

In [14]:
model.train(sentences,total_examples=len(sentences),epochs=10)

2020-08-12 00:31:06,434 : WARNING : Effective 'alpha' higher than previous training cycles
2020-08-12 00:31:06,435 : INFO : training model with 10 workers on 64647 vocabulary and 150 features, using sg=1 hs=0 sample=0.001 negative=5 window=10
2020-08-12 00:31:07,449 : INFO : EPOCH 1 - PROGRESS: at 1.13% examples, 413308 words/s, in_qsize 18, out_qsize 1
2020-08-12 00:31:08,455 : INFO : EPOCH 1 - PROGRESS: at 2.30% examples, 431601 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:31:09,472 : INFO : EPOCH 1 - PROGRESS: at 3.61% examples, 443893 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:31:10,479 : INFO : EPOCH 1 - PROGRESS: at 4.71% examples, 443711 words/s, in_qsize 20, out_qsize 0
2020-08-12 00:31:11,501 : INFO : EPOCH 1 - PROGRESS: at 5.89% examples, 439864 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:31:12,507 : INFO : EPOCH 1 - PROGRESS: at 6.85% examples, 436214 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:31:13,540 : INFO : EPOCH 1 - PROGRESS: at 7.87% examples, 431124 

2020-08-12 00:32:18,734 : INFO : EPOCH 1 - PROGRESS: at 72.92% examples, 385619 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:32:19,760 : INFO : EPOCH 1 - PROGRESS: at 73.91% examples, 385587 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:32:20,796 : INFO : EPOCH 1 - PROGRESS: at 75.03% examples, 385839 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:32:21,814 : INFO : EPOCH 1 - PROGRESS: at 76.13% examples, 385982 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:32:22,828 : INFO : EPOCH 1 - PROGRESS: at 77.32% examples, 386271 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:32:23,845 : INFO : EPOCH 1 - PROGRESS: at 78.37% examples, 386448 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:32:24,846 : INFO : EPOCH 1 - PROGRESS: at 79.39% examples, 386577 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:32:25,850 : INFO : EPOCH 1 - PROGRESS: at 80.47% examples, 386837 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:32:26,868 : INFO : EPOCH 1 - PROGRESS: at 81.62% examples, 386999 words/s,

2020-08-12 00:33:23,250 : INFO : EPOCH 2 - PROGRESS: at 40.38% examples, 400534 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:33:24,257 : INFO : EPOCH 2 - PROGRESS: at 41.51% examples, 400820 words/s, in_qsize 18, out_qsize 1
2020-08-12 00:33:25,260 : INFO : EPOCH 2 - PROGRESS: at 42.58% examples, 401171 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:33:26,271 : INFO : EPOCH 2 - PROGRESS: at 43.53% examples, 400420 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:33:27,286 : INFO : EPOCH 2 - PROGRESS: at 44.46% examples, 399059 words/s, in_qsize 20, out_qsize 0
2020-08-12 00:33:28,402 : INFO : EPOCH 2 - PROGRESS: at 45.52% examples, 398133 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:33:29,419 : INFO : EPOCH 2 - PROGRESS: at 46.58% examples, 398084 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:33:30,458 : INFO : EPOCH 2 - PROGRESS: at 47.62% examples, 397104 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:33:31,502 : INFO : EPOCH 2 - PROGRESS: at 48.66% examples, 396746 words/s,

2020-08-12 00:34:28,222 : INFO : EPOCH 3 - PROGRESS: at 8.09% examples, 390504 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:34:29,239 : INFO : EPOCH 3 - PROGRESS: at 8.95% examples, 385858 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:34:30,244 : INFO : EPOCH 3 - PROGRESS: at 9.82% examples, 381372 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:34:31,247 : INFO : EPOCH 3 - PROGRESS: at 10.95% examples, 384227 words/s, in_qsize 20, out_qsize 0
2020-08-12 00:34:32,258 : INFO : EPOCH 3 - PROGRESS: at 11.99% examples, 384496 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:34:33,269 : INFO : EPOCH 3 - PROGRESS: at 13.00% examples, 386017 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:34:34,314 : INFO : EPOCH 3 - PROGRESS: at 14.11% examples, 387303 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:34:35,330 : INFO : EPOCH 3 - PROGRESS: at 15.07% examples, 387624 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:34:36,336 : INFO : EPOCH 3 - PROGRESS: at 15.98% examples, 387362 words/s, in

2020-08-12 00:35:41,753 : INFO : EPOCH 3 - PROGRESS: at 84.48% examples, 394599 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:35:42,774 : INFO : EPOCH 3 - PROGRESS: at 85.56% examples, 394733 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:35:43,790 : INFO : EPOCH 3 - PROGRESS: at 86.77% examples, 394992 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:35:44,839 : INFO : EPOCH 3 - PROGRESS: at 87.78% examples, 395054 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:35:45,844 : INFO : EPOCH 3 - PROGRESS: at 88.87% examples, 395317 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:35:46,861 : INFO : EPOCH 3 - PROGRESS: at 89.90% examples, 395425 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:35:47,867 : INFO : EPOCH 3 - PROGRESS: at 91.03% examples, 395629 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:35:48,868 : INFO : EPOCH 3 - PROGRESS: at 92.07% examples, 395576 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:35:49,870 : INFO : EPOCH 3 - PROGRESS: at 92.95% examples, 395581 words/s,

...

2020-08-12 00:43:32,715 : INFO : EPOCH 8 - PROGRESS: at 75.18% examples, 395556 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:43:33,745 : INFO : EPOCH 8 - PROGRESS: at 76.34% examples, 395798 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:43:34,748 : INFO : EPOCH 8 - PROGRESS: at 77.50% examples, 395932 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:43:35,768 : INFO : EPOCH 8 - PROGRESS: at 78.51% examples, 395786 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:43:36,786 : INFO : EPOCH 8 - PROGRESS: at 79.59% examples, 395914 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:43:37,860 : INFO : EPOCH 8 - PROGRESS: at 80.76% examples, 395868 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:43:38,882 : INFO : EPOCH 8 - PROGRESS: at 81.86% examples, 395977 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:43:39,883 : INFO : EPOCH 8 - PROGRESS: at 82.90% examples, 396007 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:43:40,887 : INFO : EPOCH 8 - PROGRESS: at 83.88% examples, 395838 words/s,

2020-08-12 00:44:37,054 : INFO : EPOCH 9 - PROGRESS: at 42.56% examples, 402027 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:44:38,066 : INFO : EPOCH 9 - PROGRESS: at 43.63% examples, 402253 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:44:39,071 : INFO : EPOCH 9 - PROGRESS: at 44.69% examples, 402454 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:44:40,092 : INFO : EPOCH 9 - PROGRESS: at 45.71% examples, 401644 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:44:41,147 : INFO : EPOCH 9 - PROGRESS: at 46.84% examples, 401660 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:44:42,153 : INFO : EPOCH 9 - PROGRESS: at 47.99% examples, 402044 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:44:43,153 : INFO : EPOCH 9 - PROGRESS: at 49.09% examples, 402353 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:44:44,209 : INFO : EPOCH 9 - PROGRESS: at 50.27% examples, 402352 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:44:45,317 : INFO : EPOCH 9 - PROGRESS: at 51.32% examples, 402102 words/s,

2020-08-12 00:45:41,275 : INFO : EPOCH 10 - PROGRESS: at 9.10% examples, 389892 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:45:42,318 : INFO : EPOCH 10 - PROGRESS: at 10.18% examples, 391229 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:45:43,356 : INFO : EPOCH 10 - PROGRESS: at 11.32% examples, 392788 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:45:44,368 : INFO : EPOCH 10 - PROGRESS: at 12.38% examples, 391505 words/s, in_qsize 18, out_qsize 1
2020-08-12 00:45:45,372 : INFO : EPOCH 10 - PROGRESS: at 13.34% examples, 392663 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:45:46,388 : INFO : EPOCH 10 - PROGRESS: at 14.33% examples, 391182 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:45:47,398 : INFO : EPOCH 10 - PROGRESS: at 15.20% examples, 391858 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:45:48,398 : INFO : EPOCH 10 - PROGRESS: at 16.32% examples, 392820 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:45:49,399 : INFO : EPOCH 10 - PROGRESS: at 17.36% examples, 391551 

2020-08-12 00:46:54,773 : INFO : EPOCH 10 - PROGRESS: at 86.56% examples, 398848 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:46:55,860 : INFO : EPOCH 10 - PROGRESS: at 87.60% examples, 398682 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:46:56,893 : INFO : EPOCH 10 - PROGRESS: at 88.65% examples, 398780 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:46:57,952 : INFO : EPOCH 10 - PROGRESS: at 89.70% examples, 398648 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:46:58,954 : INFO : EPOCH 10 - PROGRESS: at 90.80% examples, 398817 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:46:59,976 : INFO : EPOCH 10 - PROGRESS: at 91.84% examples, 398501 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:47:00,997 : INFO : EPOCH 10 - PROGRESS: at 92.72% examples, 398436 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:47:01,999 : INFO : EPOCH 10 - PROGRESS: at 93.88% examples, 398597 words/s, in_qsize 19, out_qsize 0
2020-08-12 00:47:03,000 : INFO : EPOCH 10 - PROGRESS: at 94.88% examples, 398360

(382032776, 541118360)

### Word Embedding Vocabulary

In [9]:
words = model.wv.vocab
words

{'the': <gensim.models.keyedvectors.Vocab at 0x1a1ee56810>,
 'board': <gensim.models.keyedvectors.Vocab at 0x1a1ee56950>,
 'of': <gensim.models.keyedvectors.Vocab at 0x1a1ee56d50>,
 'directors': <gensim.models.keyedvectors.Vocab at 0x1a1ee56a50>,
 'flexshopper': <gensim.models.keyedvectors.Vocab at 0x1a1ee56ad0>,
 ',': <gensim.models.keyedvectors.Vocab at 0x1a1ee56b10>,
 'inc.': <gensim.models.keyedvectors.Vocab at 0x1a1ee56b90>,
 'a': <gensim.models.keyedvectors.Vocab at 0x1a1ee56bd0>,
 'delaware': <gensim.models.keyedvectors.Vocab at 0x1a1ee56c50>,
 'corporation': <gensim.models.keyedvectors.Vocab at 0x1a1ee56cd0>,
 '(': <gensim.models.keyedvectors.Vocab at 0x1a1ee56d10>,
 '``': <gensim.models.keyedvectors.Vocab at 0x1a1ee56e50>,
 'company': <gensim.models.keyedvectors.Vocab at 0x1a1ee56ed0>,
 "''": <gensim.models.keyedvectors.Vocab at 0x1a1ee56f50>,
 'we': <gensim.models.keyedvectors.Vocab at 0x1a1ee56090>,
 'us': <gensim.models.keyedvectors.Vocab at 0x1a1ee5a090>,
 'or': <gensim.mo

In [135]:
# Finding Word Vectors
vector = model.wv['labor']


# Most similar words i.e. predicted the next words
similar = model.wv.most_similar('harassment')
print(similar)

#Similarity scores
model.wv.similarity('trafficking', 'labor')

[('discrimination', 0.7697266340255737), ('unethical', 0.6862874031066895), ('anti-retaliation', 0.6677260994911194), ('suspected', 0.66368567943573), ('intolerance', 0.6633799076080322), ('whistleblowers', 0.6609189510345459), ('anti-corruption', 0.6478060483932495), ('anti-bribery', 0.6436290740966797), ('reprisal', 0.6319093108177185), ('retaliation', 0.6317363977432251)]


0.52068925

Under the hood, the above three snippets computes the cosine similarity between the two specified words using word vectors of each. From the scores, it makes sense that dirty is highly similar to smelly but dirty is dissimilar to clean. If you do a similarity between two identical words, the score will be 1.0 as the range of the cosine similarity score will always be between [0.0-1.0]. 

#### Words from CSAG
- human capital
- labor
- bondage
- workers
- health and safety
- diversity and inclusion
- labor relations
- safety culture
- human resources
- contractors
- fair wage
- minimum wage
- overtime
- exploitative
- injuries
- protective equipment
- workforce
- working conditions
- discrimination
- contractor
- strike
- recruitment
- Hiring


Forced labor  JB
Recruitment JB
Living wage JB
Wages JB
Health and safety JB
Collective bargaining JB
Unions JB
Labor organizing JB
pay ratio DR
pay equity DR
Grievance DR
Retaliation DR
Whistleblower DR
modern slavery DR
Incident rate DR 
Harassment DR 
TRIR (total recordable incident rate) DR
something to catch NDAs and other contractual obligations that silence worker grievances DR
something to capture anti-union / collective bargaining activity DR



### Save the Trained Word Embeddings

In [2]:
model.save('word2vec_proxy_docs.model')

#Loading
model = gensim.models.Word2Vec.load("word2vec_proxy_docs.model")

### Evaluate the word embedding model

https://towardsdatascience.com/nlp-performance-of-different-word-embeddings-on-text-classification-de648c6262b

### Try with K Means Clustering

In [117]:
from sklearn.metrics.pairwise import euclidean_distances
from sklearn import metrics
from sklearn.cluster import KMeans
import numpy as np

X = model[model.wv.vocab] #vector

features = list(model.wv.vocab) #features


kmeans = KMeans(n_clusters=8)
kmeans.fit(X)

labels = kmeans.labels_
centroids = kmeans.cluster_centers_

# # print("Labels: ")
# # print(labels)
# #print(centroids)
# # print("Score: ")
# print(kmeans.score(X))

# silhouette_score = metrics.silhouette_score(X, labels, metric='euclidean')
 
# print("Silhouette_score: ")
# print(silhouette_score)

#OPTIMAL NUMBER OF CLUSTERS: get within cluster variance
# within_cluster_variance = []
# for k in range(2,21):
#     kmeans = KMeans(n_clusters=k)
#     kmeans.fit(X)
#     within_cluster_variance.append(kmeans.inertia_)


/Users/montsieguerrero/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [116]:
import math
def optimal_number_of_clusters(wcss):
    x1, y1 = 2, wcss[0]
    x2, y2 = 20, wcss[len(wcss)-1]

    distances = []
    for i in range(len(wcss)):
        x0 = i+2
        y0 = wcss[i]
        numerator = abs((y2-y1)*x0 - (x2-x1)*y0 + x2*y1 - y2*x1)
        denominator = math.sqrt((y2 - y1)**2 + (x2 - x1)**2)
        distances.append(numerator/denominator)
    
    return distances.index(max(distances)) + 2
optimal_number_of_clusters(within_cluster_variance)

8

### Create Dataframe of Clusters

In [130]:
import pandas as pd
labels = pd.Series(labels)
features = pd.Series(features)


trial = pd.DataFrame(X)
df = pd.concat([features, labels, trial], axis = 1)

#create column names:
names = ['word', 'cluster']
for i in range(150):
    name = 'vector_' + str(i+1)
    names.append(name)
    
df.columns = names
df.head()

#subset/filter by cluster
if 'trafficking' in df[df['cluster']==1]['word'].tolist():
    print("yes")
else:
    print("no")

#ENDED HERE: need to find a way to use the words selected words 
#from CSAG and check the percentage of words found in each cluster
#create a list of all the CSAG Vocabulary

df[df['cluster']==5]['word'].tolist()

yes


['inc.',
 'corporation',
 'september',
 'november',
 'acquisitions',
 'october',
 'capital',
 'previously',
 'allied',
 'north',
 'investment',
 'incorporated',
 'harbor',
 'financial',
 'operations',
 'publicly',
 'ventures',
 'investments',
 'present',
 'january',
 'citigroup',
 'march',
 'served',
 'predecessor',
 'sterling',
 'u.s.',
 'firm',
 'operating',
 'general',
 'large',
 'global',
 'where',
 'industry',
 'leveraged',
 'lending',
 'credit',
 'banking',
 'senior',
 'several',
 'real',
 'services',
 'united',
 'states',
 'commercial',
 'retail',
 'sector',
 'finance',
 'july',
 'technology',
 'companies',
 'experience',
 'complex',
 'human',
 'financed',
 'construction',
 'energy',
 'green',
 'partners',
 'decades',
 'integration',
 'worked',
 'oil',
 'gas',
 'exploration',
 'portfolio',
 'utilities',
 'gap',
 'service',
 'development',
 '&',
 'until',
 'public',
 'blackrock',
 'group',
 'nyse',
 'member',
 'he',
 'she',
 'international',
 'leading',
 'fire',
 'advisors',
 'co

### Next Step: Need to find a way to take clusters and locate the docs with these words

### Ignore below (trial code)
### Code Trial 1 For SkipGram

In [3]:
# Author: Srijith Rajamohan

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import urllib.request
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk import word_tokenize
import sklearn
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances

torch.manual_seed(1)

CONTEXT_SIZE = 3
EMBEDDING_DIM = 10

def get_key(word_id):
    for key,val in word_to_ix.items():
        if(val == word_id):
            print(key)


def cluster_embeddings(filename,nclusters):
    X = np.load(filename)
    kmeans = KMeans(n_clusters=nclusters, random_state=0).fit(X)
    center = kmeans.cluster_centers_
    distances = euclidean_distances(X,center)

    for i in np.arange(0,distances.shape[1]):
        word_id = np.argmin(distances[:,i])
        print(word_id)
        get_key(word_id)

def read_data(file_path):
    tokenizer = RegexpTokenizer(r'\w+')
    data = urllib.request.urlopen(file_path)
    data = data.read().decode('utf8')
    tokenized_data = word_tokenize(data)
    stop_words = set(stopwords.words('english'))
    stop_words.update(['.',',',':',';','(',')','#','--','...','"'])
    cleaned_words = [ i for i in tokenized_data if i not in stop_words ]
    return(cleaned_words)


test_sentence = """Empathy for the poor may not come easily to people who never experienced it. They may blame the victims and insist their predicament can be overcome through determination and hard work.
But they may not realize that extreme poverty can be psychologically and physically incapacitating — a perpetual cycle of bad diets, health care and education exacerbated by the shaming and self-fulfilling prophecies that define it in the public imagination.
Gordon Parks — perhaps more than any artist — saw poverty as “the most savage of all human afflictions” and realized the power of empathy to help us understand it. It was neither an abstract problem nor political symbol, but something he endured growing up destitute in rural Kansas and having spent years documenting poverty throughout the world, including the United States.
That sensitivity informed “Freedom’s Fearful Foe: Poverty,” his celebrated photo essay published in Life magazine in June 1961. He took readers into the lives of a Brazilian boy, Flavio da Silva, and his family, who lived in the ramshackle Catacumba favela in the hills outside Rio de Janeiro. These stark photographs are the subject of a new book, “Gordon Parks: The Flavio Story” (Steidl/The Gordon Parks Foundation), which accompanies a traveling exhibition co-organized by the Ryerson Image Centre in Toronto, where it opens this week, and the J. Paul Getty Museum. Edited with texts by the exhibition’s co-curators, Paul Roth and Amanda Maddox, the book also includes a recent interview with Mr. da Silva and essays by Beatriz Jaguaribe, Maria Alice Rezende de Carvalho and Sérgio Burgi.
""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)

#test_sentence = read_data('https://www.gutenberg.org/files/57884/57884-0.txt')

ngrams = []
for i in range(len(test_sentence) - CONTEXT_SIZE):
    tup = [test_sentence[j] for j in np.arange(i + 1 , i + CONTEXT_SIZE + 1) ]
    ngrams.append((test_sentence[i],tup))
# print the first 3, just so you can see what they look like
#print(ngrams)


vocab = set(test_sentence)
print("Length of vocabulary",len(vocab))
word_to_ix = {word: i for i, word in enumerate(vocab)}


class SkipgramModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(SkipgramModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, 128)
        self.linear2 = nn.Linear(128, context_size * vocab_size)
        #self.parameters['context_size'] = context_size

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))  # -1 implies size inferred for that index from the size of the data
        #print(np.mean(np.mean(self.linear2.weight.data.numpy())))
        out1 = F.relu(self.linear1(embeds)) # output of first layer
        out2 = self.linear2(out1)           # output of second layer
        #print(embeds)
        log_probs = F.log_softmax(out2, dim=1).view(CONTEXT_SIZE,-1)
        return log_probs

    def predict(self,input):
        context_idxs = torch.tensor([word_to_ix[input]], dtype=torch.long)
        res = self.forward(context_idxs)
        res_arg = torch.argmax(res)
        res_val, res_ind = res.sort(descending=True)
        indices = [res_ind[i][0] for i in np.arange(0,3)]
        for arg in indices:
            print( [ (key, val) for key,val in word_to_ix.items() if val == arg ])


    def freeze_layer(self,layer):
        for name,child in model.named_children():
            print(name,child)
            if(name == layer):
                for names,params in child.named_parameters():
                    print(names,params)
                    print(params.size())
                    params.requires_grad= False

    def print_layer_parameters(self):
        for name,child in model.named_children():
                print(name,child)
                for names,params in child.named_parameters():
                    print(names,params)
                    print(params.size())

    def write_embedding_to_file(self,filename):
        for i in self.embeddings.parameters():
            weights = i.data.numpy()
        np.save(filename,weights)


losses = []
loss_function = nn.NLLLoss()
model = SkipgramModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Freeze embedding layer
#model.freeze_layer('embeddings')

for epoch in range(550):
    total_loss = 0
    #------- Embedding layers are trained as well here ----#
    #lookup_tensor = torch.tensor([word_to_ix["poor"]], dtype=torch.long)
    #hello_embed = model.embeddings(lookup_tensor)
    #print(hello_embed)
    # -----------------------------------------------------#

    model.predict('psychologically')

    for context, target in ngrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        #print(context,target)


        context_idxs = torch.tensor([word_to_ix[context]], dtype=torch.long)
        #print("Context id",context_idxs)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)
        #print(log_probs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        target_list = torch.tensor([word_to_ix[w] for w in target], dtype=torch.long)
        loss = loss_function(log_probs, target_list)
        #print(loss)

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    print(total_loss)
    losses.append(total_loss)
#print(losses)  # The loss decreased every iteration over the training data!

#Print the model layer parameters
model.print_layer_parameters()

#Predict the next word given n context words
model.predict('psychologically')
#model.write_embedding_to_file('embeddings_skipgrams.npy')
#cluster_embeddings('embeddings_skipgrams.npy',5)

Length of vocabulary 195
[('The', 107)]
[('Centre', 142)]
[('boy,', 167)]
1633.663899898529
[('The', 107)]
[('Centre', 142)]
[('boy,', 167)]
1631.1366171836853
[('The', 107)]
[('the', 93)]
[('traveling', 1)]
1628.623851299286
[('The', 107)]
[('the', 93)]
[('traveling', 1)]
1626.1251020431519
[('The', 107)]
[('the', 93)]
[('traveling', 1)]
1623.6391983032227
[('The', 107)]
[('the', 93)]
[('traveling', 1)]
1621.1656064987183
[('The', 107)]
[('the', 93)]
[('traveling', 1)]
1618.70343542099
[('The', 107)]
[('the', 93)]
[('traveling', 1)]
1616.2521777153015
[('The', 107)]
[('the', 93)]
[('and', 38)]
1613.811028957367
[('of', 40)]
[('the', 93)]
[('and', 38)]
1611.379482269287
[('of', 40)]
[('the', 93)]
[('and', 38)]
1608.957112312317
[('and', 38)]
[('the', 93)]
[('and', 38)]
1606.5434560775757
[('and', 38)]
[('the', 93)]
[('and', 38)]
1604.1373524665833
[('and', 38)]
[('the', 93)]
[('and', 38)]
1601.7386860847473
[('and', 38)]
[('the', 93)]
[('and', 38)]
1599.347312927246
[('and', 38)]
[('th

1336.1098608970642
[('and', 38)]
[('in', 3)]
[('and', 38)]
1333.6661386489868
[('and', 38)]
[('in', 3)]
[('and', 38)]
1331.2112157344818
[('and', 38)]
[('in', 3)]
[('and', 38)]
1328.7453410625458
[('and', 38)]
[('in', 3)]
[('and', 38)]
1326.2690994739532
[('and', 38)]
[('in', 3)]
[('and', 38)]
1323.7835309505463
[('and', 38)]
[('in', 3)]
[('and', 38)]
1321.2871911525726
[('and', 38)]
[('in', 3)]
[('and', 38)]
1318.7801744937897
[('and', 38)]
[('in', 3)]
[('and', 38)]
1316.263379573822
[('and', 38)]
[('in', 3)]
[('and', 38)]
1313.7360908985138
[('and', 38)]
[('in', 3)]
[('and', 38)]
1311.1976754665375
[('and', 38)]
[('in', 3)]
[('and', 38)]
1308.6489536762238
[('and', 38)]
[('in', 3)]
[('and', 38)]
1306.0887959003448
[('and', 38)]
[('in', 3)]
[('and', 38)]
1303.5179591178894
[('and', 38)]
[('in', 3)]
[('and', 38)]
1300.936121225357
[('and', 38)]
[('in', 3)]
[('and', 38)]
1298.3438413143158
[('and', 38)]
[('in', 3)]
[('and', 38)]
1295.7412915229797
[('and', 38)]
[('in', 3)]
[('and', 38)]

955.4474303722382
[('and', 38)]
[('in', 3)]
[('incapacitating', 121)]
952.4395378828049
[('and', 38)]
[('in', 3)]
[('incapacitating', 121)]
949.4378402233124
[('and', 38)]
[('in', 3)]
[('incapacitating', 121)]
946.4438147544861
[('and', 38)]
[('in', 3)]
[('incapacitating', 121)]
943.455855846405
[('and', 38)]
[('in', 3)]
[('incapacitating', 121)]
940.476300239563
[('and', 38)]
[('in', 3)]
[('incapacitating', 121)]
937.503947019577
[('and', 38)]
[('in', 3)]
[('incapacitating', 121)]
934.5387010574341
[('and', 38)]
[('in', 3)]
[('incapacitating', 121)]
931.5823435783386
[('and', 38)]
[('in', 3)]
[('incapacitating', 121)]
928.6333845853806
[('and', 38)]
[('in', 3)]
[('incapacitating', 121)]
925.6922042369843
[('and', 38)]
[('in', 3)]
[('incapacitating', 121)]
922.7598987817764
[('and', 38)]
[('in', 3)]
[('incapacitating', 121)]
919.8355667591095
[('and', 38)]
[('in', 3)]
[('incapacitating', 121)]
916.9203156232834
[('and', 38)]
[('in', 3)]
[('incapacitating', 121)]
914.0132431983948
[('an

692.1064677238464
[('and', 38)]
[('physically', 88)]
[('incapacitating', 121)]
690.3403713703156
[('and', 38)]
[('physically', 88)]
[('incapacitating', 121)]
688.5859253406525
[('and', 38)]
[('physically', 88)]
[('incapacitating', 121)]
686.8436834812164
[('and', 38)]
[('physically', 88)]
[('incapacitating', 121)]
685.1131255626678
[('and', 38)]
[('physically', 88)]
[('incapacitating', 121)]
683.394660949707
[('and', 38)]
[('physically', 88)]
[('incapacitating', 121)]
681.687496304512
[('and', 38)]
[('physically', 88)]
[('incapacitating', 121)]
679.9931452274323
[('and', 38)]
[('physically', 88)]
[('incapacitating', 121)]
678.3094831705093
[('and', 38)]
[('physically', 88)]
[('incapacitating', 121)]
676.6384865045547
[('and', 38)]
[('physically', 88)]
[('incapacitating', 121)]
674.9787582159042
[('and', 38)]
[('physically', 88)]
[('incapacitating', 121)]
673.3302702903748
[('and', 38)]
[('physically', 88)]
[('incapacitating', 121)]
671.6943442821503
[('and', 38)]
[('physically', 88)]
[

562.3236879110336
[('and', 38)]
[('physically', 88)]
[('incapacitating', 121)]
561.4935898780823
[('and', 38)]
[('physically', 88)]
[('incapacitating', 121)]
560.6693658828735
[('and', 38)]
[('physically', 88)]
[('incapacitating', 121)]
559.8505252599716
[('and', 38)]
[('physically', 88)]
[('incapacitating', 121)]
559.0381718873978
[('and', 38)]
[('physically', 88)]
[('incapacitating', 121)]
558.2317596673965
[('and', 38)]
[('physically', 88)]
[('incapacitating', 121)]
557.4302178621292
[('and', 38)]
[('physically', 88)]
[('incapacitating', 121)]
556.6352490186691
[('and', 38)]
[('physically', 88)]
[('incapacitating', 121)]
555.8457530736923
[('and', 38)]
[('physically', 88)]
[('incapacitating', 121)]
555.0619252920151
[('and', 38)]
[('physically', 88)]
[('incapacitating', 121)]
554.283113360405
[('and', 38)]
[('physically', 88)]
[('incapacitating', 121)]
553.5105181932449
[('and', 38)]
[('physically', 88)]
[('incapacitating', 121)]
552.7424221038818
[('and', 38)]
[('physically', 88)]
